## THE ADVISORY FILE
#mock data

## loading the datasets

In [ ]:
#LOADING THE DATASETS
df= pd.read_csv('crop_recommendation.csv')
def get_location_name(lat, lon):
    api_key = "d16ca93ca7c89a66afa56bd31a522391"
    url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}&units=metric"

    try:
        response = requests.get(url, timeout=5)
        if response.status_code == 200:
            data = response.json()
            location_name = data.get('name', 'Unknown Location')
            return location_name
        else:
            print("Error fetching data:", response.status_code, response.text)
            return 'Unknown Location'
    except requests.exceptions.RequestException as e:
        print("Network error:", e)
        return  'Unknown Location'

print(df.head)

## months that favor the crops

In [ ]:

crop_months = {
   'rice': [3, 4, 5, 10, 11], 'maize': [3, 4, 9, 10], 'chickpea': [1, 2, 11, 12],
    'kidneybeans': [5, 6], 'pigeonpeas': [6, 7], 'mothbeans': [7, 8],
    'mungbean': [8, 9], 'blackgram': [6, 7, 8], 'lentil': [1, 2, 11, 12],
    'pomegranate': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 'banana': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    'mango': [1, 2, 3], 'grapes': [11, 12, 1, 2], 'watermelon': [1, 2, 3],
    'muskmelon': [1, 2, 3], 'apple': [1, 2, 11, 12], 'orange': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    'papaya': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 'coconut': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    'cotton': [5, 6, 7, 8], 'jute': [3, 4, 5], 'coffee': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
}

print("crop months set!")


### risk assessment layer

In [ ]:
def risk_assessment ( humidity, rainfall, temperature):
    risk_score = 0
    factors = []

    #fungal attack is high when humidity is high
    if humidity > 80 and 20 <= temperature<=30:
        risk_score += 60
        factors.append("Heavy rainfall and warmth: Your crops are at risk of fungal blight.")
    if rainfall>200:
        risk_score += 20
        factors.append("Heavy rainfall: Monitor for water-borne pests.")
    if rainfall <20 and temperature >35:
        risk_score +=70
        factors.append("Extreme heat and low rainfall amounts: Your crops are at high risk of wilting")
    #categorixing the risk
    level = "low"
    if risk_score >70 : level = "high"
    elif risk_score > 40: level = 'medium'


    return {"level": level, "score": risk_score, "warnings": factors}


### monthly data usage

In [ ]:
import requests
import pandas as pd
import datetime

def openmeteo_monthly_weather(lat, lon, year=None):
    today = datetime.datetime.now()
    if year is None:
        year = today.year
    
    #get the whole year for simplicity, then take the last month
    start_date = f"{year}-01-01"
    end_date = f"{year}-12-31"

    url = (
        f"https://archive-api.open-meteo.com/v1/archive?"
        f"latitude={lat}&longitude={lon}&"
        f"start_date={start_date}&end_date={end_date}&"
        f"hourly=temperature_2m,relativehumidity_2m,precipitation&timezone=Africa/Nairobi"
    )

    response = requests.get(url, timeout=10)
    if response.status_code != 200:
        print("Error fetching Open-Meteo data, using default mock values")
        return 25.0, 70.0, 50.0, "Unknown Location"

    data = response.json()

    df_weather = pd.DataFrame({
        'time': pd.to_datetime(data['hourly']['time']),
        'temperature': data['hourly']['temperature_2m'],
        'humidity': data['hourly']['relativehumidity_2m'],
        'rainfall': data['hourly']['precipitation']
    })

    df_weather.set_index('time', inplace=True)

    # Monthly aggregation
    monthly_data = df_weather.resample('ME').agg({
        'temperature': 'mean',
        'humidity': 'mean',
        'rainfall': 'sum'
    })

    latest = monthly_data.iloc[-1]

    return latest['temperature'], latest['humidity'], latest['rainfall']
print('successfully converted data to monthly')

## training and testing the datasets

In [ ]:

#TRAINING
X= df.drop('label', axis=1) # X = FEATURE (QUESTION / INPUTS)
y=df['label']    #Y = TARGET (ANSWER)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.head)

#training only the training set
model =RandomForestClassifier(n_estimators=100) 
model.fit(X_train, y_train)

#evaluating the test
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print('Model training complete!')
print(f"RANDOM FOREST ACCURACY SCORE: {accuracy * 100:.2f}%")


##  The advisory logic

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
import joblib
import datetime


def farmer_advice(lat, lon, soil_type, n, p, k, ph):
    location_name = get_location_name(lat, lon)
    current_month = datetime.datetime.now().month

    temperature, humidity, rainfall, location_name = openmeteo_monthly_weather(lat, lon)
    input_df = pd.DataFrame([{
        'N': n,
        'P': p,
        'K': k,
        'temperature': temperature,
        'humidity': humidity,
        'ph': ph,
        'rainfall': rainfall
    }])
    #prediction
    prediction = model.predict(input_df)
    month_name= datetime.datetime.now().strftime("%B")

    risk_info = risk_assessment(temperature, humidity, rainfall)

    advice = {
        "location": f"{location_name} ({lat}, {lon})",
        "recommended_crop": prediction[0],
        "risk_level": risk_info["level"],
        "soil_analysis": f"Type: {soil_type}, PH: {ph}",
        "alerts": risk_info["warnings"],
        "message":f"We are currently in the month of {month_name}. {location_name}'s soil type is {soil_type}. {location_name} is currently at {temperature} degrees, {rainfall}mm of rainfall and a humidity of {humidity}%. Based on that, recommend planting {prediction[0]}, Watch on risk levels which is currently {risk_info['level']}"
    }
    return advice
farmer_advice(-0.42, 36.95, "loamy", 90, 40, 40, 6.5)
    